## Importing libraries

In [65]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np


## Extracting active and inactive for 'LLC','INC','LLP','Corp' and 'LP' and saving each as .csv file


In [67]:



keys = ['LLC','INC','LLP','Corp','LP']


active_id = "MainContent_rdbActive"
inactive_id = "MainContent_rdbInactive"


def get_all_active(current):  
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select = Select(dropdown_element)
    select.select_by_visible_text("All items")
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text
        header.append(title)
    companies = pd.DataFrame(columns = header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(companies)
        companies.loc[length] = row
    companies['status'] = 'active'
    companies['entity_name'] = current
    companies.to_csv(f"companies_active_{current}.csv")
    print(f'All Actives for {current} ssuccessfully scraped')
    driver.quit()


def get_all_inactive(current):  
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')

    button = driver.find_element(By.ID, inactive_id)


    button.click()
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select = Select(dropdown_element)
    select.select_by_visible_text("All items")
    
    #text_field.send_keys(Keys.RETURN)
    
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text
        header.append(title)
    companies = pd.DataFrame(columns = header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(companies)
        companies.loc[length] = row
    companies['status'] = 'inactive'
    companies['entity_name'] = current
    companies.to_csv(f"companies_inactive_{current}.csv")
    print(f'All Inactives for {current} successfully scraped')
    driver.quit()
    companies

for i in keys:
    get_all_inactive(i)

for i in keys:
    get_all_active(i)
    




All Inactives for LLC successfully scraped
All Inactives for INC successfully scraped
All Inactives for LLP successfully scraped
All Inactives for Corp successfully scraped
All Inactives for LP successfully scraped
All Actives for LLC ssuccessfully scraped
All Actives for INC ssuccessfully scraped
All Actives for LLP ssuccessfully scraped
All Actives for Corp ssuccessfully scraped
All Actives for LP ssuccessfully scraped


## Merging the extracted .csv files

In [23]:
import pandas as pd
import glob
import os

csv_files = glob.glob(os.path.join(os.getcwd(), "*.csv"))

df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.drop_duplicates(inplace=True)

#combined_df.to_csv("combined_output.csv", index=False)



,Unnamed: 0,Entity Name,ID Number,NAICS,Inactive status,Address,status,entity_name
370,27,L.P.Q. Inc.,4805,,Revoked Entity,", USA",inactive,LP
145,3,"CORP BROTHERS, INCORPORATED **CONVERTED TO A N...",4861,,Conversion,"88 NIANTIC AVENUE PROVIDENCE, RI 02907 USA",inactive,Corp
155,13,"Corporate Air Charter, inc.",4862,,Revoked Entity,"38 ANDREWS AVENUE, APT.A WEST WARWICK, RI 028...",inactive,Corp
160,18,"Corporate Associates, Limited",4863,,Revoked Entity,", USA",inactive,Corp
152,10,"CORPORATE & ESTATE ANALYST, INCORPORATED",4864,,Revoked Entity,", USA",inactive,Corp
...,...,...,...,...,...,...,...,...
43,43,"CORPORATE TECHNOLOGIES, INC.",1767222,,,"78 BLANCHARD RD STE 206 BURLINGTON, MA 01803 ...",active,Corp
99,1,L&L Paws,1767793,,,"47 WOOD AVE SUITE 2BARRINGTON, RI 02806 USA",active,LLP
65,6,INCENTER INSURANCE SOLUTIONS LLC,1770400,,,"850 NEW BURTON ROAD SUITE 201DOVER, DE 19904 ...",active,INC
113,11,"LP Coaching, LLC",1772536,,,"43 ORCHARD AVE SOUTH KINGSTOWN, RI 02879 USA",active,LP


## Scraping per identification number page from "000000001" to "001775776"

### Date of incorporation not qualification

### The exact name: <span style="color: green;">✅</span>

### The fictitious name and filed date: <span style="color: red;">❌</span> Not Available

### Entity type: <span style="color: green;">✅</span>

### Identification Number: <span style="color: green;">✅</span>

### Date of Incorporation in Rhode Island: <span style="color: green;">✅</span>

### Effective Date: <span style="color: green;">✅</span>

### The location of the Principal Office: <span style="color: green;">✅</span>

### Agent Resigned: <span style="color: green;">✅</span>

### Address Maintained: <span style="color: green;">✅</span>

### The name and address of the Registered Agent: <span style="color: green;">✅</span>

### The Officers and Directors of the Corporation (if INC or Corp): <span style="color: green;">✅</span>

### The name and business address of each General Partner (if LP): <span style="color: green;">✅</span>

### The name and business address of each Manager (if LLC): <span style="color: green;">✅</span>

### The name and business address of each Partner who is authorized to execute, acknowledge, deliver, and record any recordable instrument purporting to affect any interest in real property: <span style="color: red;">❌</span> Not Available

### The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue: <span style="color: green;">✅</span>

### Purpose: <span style="color: green;">✅</span>


In [70]:
id_num = 1723944
exact_names = []
entity_types = []
identification_numbers = []
date_of_incorporations = []
effective_dates = []
loc_principal_offices = []
agent_resigned = []
address_maintained = []
name_address_reg_agents = []
off_dir_corps = []
name_business_add_GPs = []
LLC_managers = []
stocks = []
purposes = []
NIACS = []

# service = Service(executable_path=r'C:\Users\H i - G E O R G E\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')
# options = Options()
# ua = UserAgent()
# userAgent = ua.random
# options.add_argument(f'user-agent={userAgent}')

# driver = webdriver.Chrome(service=service,options=options)
# driver.get(f'https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN={id_num}&SEARCH_TYPE=3')

# time.sleep(2)
# page = driver.page_source
url = f'https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN={id_num}&SEARCH_TYPE=3'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)

soup = BeautifulSoup(page.content, "html.parser")

#exact_name
if soup.find('span',attrs={'id': "MainContent_lblExactEntityName"}) is not None:
    exact_name = soup.find('span',attrs={'id': "MainContent_lblExactEntityName"}).text
    exact_names.append(exact_name)
else:
    exact_names.append('None')
    
#entity_type
if soup.find('span',attrs={'id': "MainContent_lblEntityType"}) is not None:
    entity_type = soup.find('span',attrs={'id': "MainContent_lblEntityType"}).text
    entity_types.append(entity_type)
else:
    entity_types.append('None')

#identification number
if soup.find('span',attrs={'id': "MainContent_lblIDNumber"}) is not None:
    identification_number = soup.find('span',attrs={'id': "MainContent_lblIDNumber"}).text.replace('Identification Number: ','')
    identification_numbers.append(identification_number)
else:
    identification_numbers.append('None')

#date of incorporation
if soup.find('span',attrs={'id': "MainContent_lblOrganisationDate"}) is not None:
    date_of_incorporation = soup.find('span',attrs={'id': "MainContent_lblOrganisationDate"}).text
    date_of_incorporations.append(date_of_incorporation)
else:
    date_of_incorporations.append('None')

#effective date
if soup.find('span',attrs={'id': "MainContent_lblEffectiveDate"}) is not None:
    effective_date = soup.find('span',attrs={'id': "MainContent_lblEffectiveDate"}).text
    effective_dates.append(effective_date)
else:
    effective_dates.append('None')

#location of principal office
if soup.find('table',attrs={'id': "MainContent_tblPrincipleOffice"}) is not None:
    loc_principal_office = soup.find('table',attrs={'id': "MainContent_tblPrincipleOffice"}).text.replace('\xa0','').replace('\n','').replace('The location of the Principal Office:','').replace('Address:','').replace('City or Town, State, Zip, Country:',',')
    loc_principal_office = re.sub(r'\s+', ' ', loc_principal_office)
    loc_principal_offices.append(loc_principal_office)
else:
    loc_principal_offices.append('None')


#agent resigned
if soup.find('span',attrs={'id': "MainContent_lblResidentAgentFlag"}) is not None:
    agent_resign = soup.find('span',attrs={'id': "MainContent_lblResidentAgentFlag"}).text
    agent_resigned.append(agent_resign)
else:
    agent_resigned.append('None')
    

#address maintained
if soup.find('span',attrs={'id': "MainContent_lblConsentFlag"}) is not None:
    address_maintain = soup.find('span',attrs={'id': "MainContent_lblConsentFlag"}).text
    address_maintained.append(address_maintain)
else:
    address_maintained.append('None')


#The name and address of the Registered Agent
if soup.find('table',attrs={'id': "MainContent_tblResident"}) is not None:
    name_address_reg_agent = soup.find('table',attrs={'id': "MainContent_tblResident"}).text.replace('Name:','').replace('\n','').replace('The name and address of the Registered Agent:','').replace('Address:',',').replace('City or Town, State, Zip, Country:',',')
    name_address_reg_agent = re.sub(r'\s+', ' ', name_address_reg_agent)
    name_address_reg_agents.append(name_address_reg_agent)
else:
    name_address_reg_agents.append('None')


#The Officers and Directors of the Corporation
if soup.find('table',attrs={'id': "MainContent_tblOfficers"}) is not None:
    off_dir_corp = soup.find('table',attrs={'id': "MainContent_tblOfficers"}).text.replace('TitleIndividual NameAddress','').replace('\n','').replace('The Officers and Directors of the Corporation:','').replace('Address:',',').replace('City or Town, State, Zip, Country:',',')
    off_dir_corp = re.sub(r'\s+', ' ', off_dir_corp)
    off_dir_corps.append(off_dir_corp)
else:
    off_dir_corps.append('None')
    
#The name and business address of each General Partner
if soup.find('table',attrs={'id': "MainContent_grdManagers"}) is not None:
    name_business_add_GP = soup.find('table',attrs={'id': "MainContent_grdManagers"}).text.replace('TitleIndividual nameAddress','').replace('\n','')
    name_business_add_GP = re.sub(r'\s+', ' ', name_business_add_GP)
    name_business_add_GPs.append(name_business_add_GP)
else:
    name_business_add_GPs.append('None')



#The limited liability company is to be managed by its Managers
if soup.find('table',attrs={'id': "MainContent_tblManagers"}) is not None:
    LLC_manager = soup.find('table',attrs={'id': "MainContent_tblManagers"}).text.replace('The limited liability company is to be managed by its ManagersThe name and business address of each Manager:','').replace('\n','').replace('TitleIndividual nameAddress','').replace('The limited liability company is to be managed by its MembersThe name and business address of each Manager:','')
    LLC_manager = re.sub(r'\s+', ' ', LLC_manager)
    LLC_managers.append(LLC_manager)
else:
    LLC_managers.append('None')



#The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue
if soup.find('table',attrs={'id': "MainContent_tblStocks"}) is not None:
    stock = soup.find('table',attrs={'id': "MainContent_tblStocks"}).text.replace('The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue:','').replace('\n','').replace('TitleIndividual nameAddress','')
    stock = re.sub(r'\s+', ' ', stock)
    stocks.append(stock)
else:
    stocks.append('None')


#Purpose
if soup.find('table',attrs={'id': "MainContent_tblComments"}) is not None:
    purpose = soup.find('table',attrs={'id': "MainContent_tblComments"}).text.replace('Purpose:','').replace('\n','')
    purpose = re.sub(r'\s+', ' ', purpose)
    purposes.append(purpose)
else:
    purposes.append('None')



#North American Industry Classification System Code(NAICS)
if soup.find('table',attrs={'id': "MainContent_tblNIACS"}) is not None:
    NIAC = soup.find('table',attrs={'id': "MainContent_tblNIACS"}).text.replace('North American Industry Classification System Code(NAICS):','').replace('\n','')
    NIAC = re.sub(r'\s+', ' ', NIAC)
    NIACS.append(NIAC)
else:
    NIACS.append('None')


#driver.quit()
address_maintained

['Y']

In [35]:
loc_principal_offices

['\n\n\nThe location of the Principal Office:\n\n\n\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\n\nAddress:\n \n\n\nCity or Town, State, Zip, Country:\n\n\n                            \xa0\n                            \n                            \xa0\n                            \n                            \xa0\n                            USA\n\n\n']

In [24]:
scrape the open data of the State of Rhode Island for all registered companies (3 different types)

Website: https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx

SEARCH CRITERIA:
Choose
"Active" and then "Inactive" after Scraping Active companies
1. "Search by entity name":
Search for:
LLC: Scrape all the companies
INC: Scrape all the companies
LLP: Scrape all the companies
Corp: Scrape all the companies
LP: Scrape all the companies

"Inactive" (after finish Scraping for Active companies), repeat the process with Inactive companies.
Note: There might be duplicate companies between the searches, you could delete the duplicates.

2. Or you can "Search by Identification Number":
Try from "000000001" to "001775776"

Fields:
-The exact name:

-The fictitious name and filed date (if have):

-Entity type:

-Identification Number:

-Date of Qualification in Rhode Island:

-Effective Date:

-The location of the Principal Office:

-Agent Resigned:

-Address Maintained:

-The name and address of the Registered Agent:

-The Officers and Directors of the Corporation (if INC or Corp):
1.Title:
2.Individual Name:
3.Address:

-The name and business address of each General Partner (if LP):
1.Title:
2.Individual Name:
3.Address:

-The limited liability company is to be managed by its Managers (if LLC)
-The name and business address of each Manager (if LLC):
1.Title:
2.Individual Name:
3.Address:

-The name and business address of each Partner who is authorized to execute, acknowledge, deliver, and record any recordable instrument purporting to affect any interest in real property (if LLP):
1.Title:
2.Individual Name:
3.Address:

-The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue (if have):

-Purpose:

-North American Industry Classification System Code(NAICS):

-View filings for this business entity:

Output:
Microsoft Excel file
Source Code (Python source code file must also be delivered to us)

SyntaxError: invalid decimal literal (257005525.py, line 44)

In [45]:
import re

# Sample list with excess white space in the middle of the string
address_list = ['205 AIRPORT ROAD ,WARWICK,                                                         RI                                                        02889-1029                                                        USA']

# Function to clean the string by removing excess white space
def clean_string(s):
    return re.sub(r'\s+', ' ', s)

# Apply the function to each string in the list
cleaned_list = [clean_string(s) for s in address_list]

# Print the cleaned list
print(cleaned_list)


['205 AIRPORT ROAD ,WARWICK, RI 02889-1029 USA']
